In [1]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

In [ ]:
model = YOLO("yolo11l.pt")

video_path = "samples/vietnam.mp4"
cap = cv2.VideoCapture(video_path)

In [ ]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

video_name = video_path.split("/")[-1]
output_path = f"run/{video_name.split('.')[0]}_tracked.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


In [ ]:
track_history = defaultdict(lambda: [])

while cap.isOpened():
    success, frame = cap.read()
    
    if success:
        results = model.track(frame, persist=True, show=False)
        
        boxes = results[0].boxes.xywh.cpu()
        try:
            track_ids = results[0].boxes.id
            if track_ids is not None:
                track_ids = track_ids.int().cpu().tolist()
                
            else:
                track_ids = []
        except AttributeError:
            track_ids = []
        
        annotated_frame = results[0].plot()
        if track_ids:
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                
                if len(track) > 120:
                    track.pop(0)
                    
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(
                    annotated_frame, [points], 
                    isClosed=False, color=(230, 230, 230),
                    thickness=4
                )
                
        out.write(annotated_frame)
    
    else:
        break
    
cap.release()
out.release()
print(f"Video has been saved to {output_path}")